Wilson's personal notebook

Metric: Transportation

In [ ]:
# import libraries
import geopandas as gpd
import pandas as pd

# read data
neighborhoods = gpd.read_file('https://data.wprdc.org/dataset/e672f13d-71c4-4a66-8f38-710e75ed80a4/resource/4af8e160-57e9-4ebf-a501-76ca1b42fc99/download/pittsburghpaneighborhoods-.geojson')